In [9]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

In [10]:

def is_yellow(pixel):
    return pixel[0] == 255 and pixel[1] == 255 and pixel[2] == 0

def is_green(pixel):
    return pixel[0] == 0 and pixel[1] == 255 and pixel[2] == 0

def bfs(image, x, y, target_color, new_color):
    rows, cols = image.shape[:2]
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    queue = [(x, y)]
    
    while queue:
        cx, cy = queue.pop()
        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy
            if 0 <= nx < rows and 0 <= ny < cols:
                if (target_color == 'yellow' and not is_green(image[nx, ny])) or (target_color == 'green' and not is_yellow(image[nx, ny])):
                    break
        else:
            image[x, y] = new_color
            return

def process_image(image_path):
    image = Image.open(image_path)
    image_np = np.array(image)
    
    rows, cols = image_np.shape[:2]
    new_image_np = image_np.copy()

    for x in range(rows):
        for y in range(cols):
            if is_yellow(image_np[x, y]):
                bfs(new_image_np, x, y, 'yellow', (0, 255, 0))
            elif is_green(image_np[x, y]):
                bfs(new_image_np, x, y, 'green', (255, 255, 0))

    new_image = Image.fromarray(new_image_np)
    new_image.save(image_path)

def process_folder(folder_path):
    for root, _, files in tqdm(os.walk(folder_path)):
        
        for file in files:
            #print(file)
            if file.startswith('predicted_mask') and file.endswith('.jpg'):
                process_image(os.path.join(root, file))

result_folder = './result'
test_folder = os.path.join(result_folder, 'test')
val_folder = os.path.join(result_folder, 'val')

process_folder(test_folder)
process_folder(val_folder)


1it [02:32, 152.07s/it]
1it [02:30, 150.20s/it]
